# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. openai API KEY 발급
2. google colab - secret - `OPENAI_API_KEY` 등록

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "나는 차은우야"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=0.1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반갑습니다. 어떤 이야기를 나누고 싶으신가요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력데이터/예시
4. Output Indicator 출력 지시사항

## 기사 제목 교정
- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사 제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결히 정리하세요.

### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사의 원래 제목]
- 교정 제목: [기사의 교정된 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감, 반기는 내연차회사"

### Extra Instructions ###
- 제목이 너무 길면 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.
"""

user_message = f"""
다음 기사제목을 간결히 수정해주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: "테이의 FM 개꿀 라디오 방송에 주목해주세요."
- 교정 제목: "테이의 FM 라디오 방송에 주목하세요."


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사 제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결히 정리하세요.

    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사의 원래 제목]
    - 교정 제목: [기사의 교정된 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감, 반기는 내연차회사"

    ### Extra Instructions ###
    - 제목이 너무 길면 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.
    """

    user_message = f"""
    다음 기사제목을 간결히 수정해주세요.

    {title_before}
    """

    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": system_message
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": user_message
            }
          ]
        }
      ],
      response_format={
        "type": "text"
      },
      temperature=temperature,
      max_completion_tokens=2048,
      top_p=top_p,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길'
output = correct_news_title(title_before)
print(output)

- 원래 제목: "주말 미친 폭우 예상, 모두들 무사하시길"
- 교정 제목: "주말 폭우 예상, 안전에 유의하세요"


## 연애코치 ReAct

In [3]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:

    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": system_message
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": user_message
            }
          ]
        }
      ],
      response_format={
        "type": "text"
      },
      temperature=temperature,
      max_completion_tokens=2048,
      top_p=top_p,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

In [5]:
print(dating_coach('저 이번 주말에 여자친구와 100일인데, 어떡하죠?'))

1. 상황분석:
여자친구와의 100일 기념일은 관계의 중요한 이정표입니다. 이 특별한 날을 기념하기 위해서는 감정적이고 의미 있는 접근이 필요합니다. 무엇보다 여자친구가 특별하게 느끼도록 배려하는 것이 중요합니다. 현재 어떤 기념일을 계획중인지, 예산은 어느 정도인지, 여자친구의 취향은 무엇인지를 고려해야 합니다.

2. 행동계획:
- 기념일에 어떤 이벤트를 할지 결정합니다. (예: 특별한 식사, 서프라이즈 이벤트, 편지)
- 여자친구가 좋아하는 장소나 활동을 선택합니다.
- 작은 선물이나 손편지를 준비하여 감정적인 메시지를 전달합니다.
- 식사 또는 활동 예약이 필요하다면 미리 준비합니다.

3. 실행:
- 100일 기념일 전날 또는 당일 아침에 여자친구에게 미리 특별한 계획을 살짝 암시해 흥미를 유도합니다.
- 타이밍을 잘 맞춰 기념일을 축하할 수 있는 분위기를 조성합니다.
- 준비한 선물과 편지를 전달하며 진정한 마음을 표현합니다.
- 하루를 그녀와 함께 즐기고, 함께하는 순간을 소중히 여깁니다. 

이런 계획을 통해 100일 기념일이 특별한 추억으로 남기를 바랍니다!


## 냉털마스터 ReAct
- 사용자가 냉장고에 남아있는 음식 재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Resoning + Action 기법을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [18]:
def fridge_master(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    [시스템 지시사항]
    당신은 요리 전문가이자 창의적인 레시피 개발자입니다. 사용자가 제공한 냉장고 속 재료 목록을 바탕으로 만들 수 있는 요리를 제안하십시오. 단순히 흔한 요리를 나열하는 것이 아니라, 각 재료의 조합 가능성을 판단하고, 다음 기준을 만족하도록 논리적으로 추론한 뒤 조리법을 제시하십시오.

    [제안 조건]
    - Reasoning 과정을 먼저 보여주세요. (어떤 조합이 가능한지, 어떤 음식으로 응용 가능한지)
    - Action: 최종적으로 하나의 요리를 선택하고, 그 이유와 함께 간단한 조리법을 안내해주세요.
    - 비용 효율성, 시간, 맛의 균형을 고려해주세요.
    - 일반적인 요리가 아니라 창의적인 활용 아이디어도 좋습니다 (예: 간식, 소스, 반찬, 퓨전 음식 등).
    - 음식 이름은 명확하게 표기하고, 한국어로 답변해주세요.

    [확장 지시사항]
    - 사용자가 추가 질문(예: “도시락용으로 바꿔줘”, “더 빠른 방법 있어?”, “비건 버전 가능해?” 등)을 하면, 앞서 추천한 요리를 바탕으로 적절한 대안을 제시하거나 조리법을 조정해 대응하십시오.
    - 앞의 Reasoning과 Action 흐름을 유지하되, 후속 대화 요청에 유연하고 구체적으로 반응하십시오.

    [예시 입력]
    재료: 두부, 고추장, 양파, 상한 우유, 식빵 2조각, 달걀 1개, 케찹, 시금치 약간

    [출력 형식 예시]

    [Reasoning]
    - 두부 + 고추장 + 양파 → 매콤 두부조림 가능
    - 식빵 + 달걀 + 시금치 → 프렌치토스트 변형 가능
    - 상한 우유는 폐기 필요
    - 케찹은 소스로 활용 가능

    [Action]
    ▶ 추천 요리: ‘매콤 시금치 두부 오픈토스트’
    1. 두부를 으깨 고추장과 케찹으로 양념 후 볶고,
    2. 시금치와 함께 식빵 위에 올려 오븐에 살짝 굽습니다.
    - 간단하면서도 영양 균형이 좋고, 재료 활용도가 높습니다.

    **질문이 있다면 언제든 물어보세요. (ex. 더 간단한 버전 없을까?, 비건으로도 가능해?, 다른 종류의 요리는 없을까?)**
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": system_message
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": user_message
            }
          ]
        }
      ],
      response_format={
        "type": "text"
      },
      temperature=temperature,
      max_completion_tokens=2048,
      top_p=top_p,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

In [19]:
from IPython.display import Markdown
user_input = '내 냉장고에는 마요네즈, 계란, 파김치, 매운멸치볶음, 수박, 햇반, 신라면, 상추, 삼겹살, 오렌지주스가 있는데 양식으로 만들 수 있을까?'
display(Markdown(fridge_master(user_input)))

[Reasoning]
- 계란 + 마요네즈 → 크림 소스 또는 스크램블 에그 가능
- 파김치 + 삼겹살 → 김치볶음 가능
- 매운멸치볶음 → 반찬 역할, 혹은 크루통으로 변형 가능
- 상추 → 샐러드 또는 랩으로 활용 가능
- 햇반 + 계란 → 볶음밥 변형 가능
- 수박 → 디저트나 아보카도로 변형 가능
- 신라면 → 즉석 면 요리 가능
- 오렌지주스 → 드레싱 또는 음료 활용

이 재료들로 양식 요리를 만들어보면, 샐러드와 함께 제공할 수 있는 메인 디쉬 구성이 가능할 것 같습니다. 

[Action]
▶ 추천 요리: ‘삼겹살 오렌지 드레싱 샐러드’
1. 삼겹살을 팬에서 바삭하게 구워주세요.
2. 상추를 씻어 먹기 좋은 크기로 찢고, 파김치를 슬라이스합니다.
3. 오렌지주스를 약간의 소금과 후추로 드레싱을 만들어 상추와 파김치에 버무립니다.
4. 구운 삼겹살 위에 샐러드를 올리고, 매운멸치볶음을 크루통처럼 뿌려주면 완성됩니다.

- 이 요리는 신선함과 고소한 맛이 조화를 이루며, 오렌지 드레싱이 여름에 어울리는 상큼함을 더해줍니다. 또한 양식으로 변형된 한국 요리의 퓨전 느낌도 느낄 수 있습니다.

**질문이 있다면 언제든 물어보세요. (ex. 더 간단한 버전 없을까?, 비건으로도 가능해?, 다른 종류의 요리는 없을까?)**

## 면접 질문 생성 JSON 출력

In [21]:
def job_interveiw(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당 분야의 으뜸가는 면접관입니다.
    매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

    << 지시사항 >>
    - 사용자가 제출한 채용공고의 내용을 바탕으로 예상 면접 질문과 답변을 작성해 주세요.
    - 하드 스킬과 소프트 스킬 두개의 섹션으로 나누어 작성해주세요.
    - 각 스킬별로 질문/답변을 3개씩 만들어주세요.

    << 출력형식 >>
    출력은 json 형식으로만 반환되어야 합니다.

    {{
      "hard_skill": [
        {{
          "question": "",
          "anser": "",
        }}
      ],
      "soft_skill": [
        {{
          "question": "",
          "anser": "",
        }}
      ]
    }}
    """

    user_message = f"""
    채용공괴
    {job_posting}
    """

    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": system_message
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": user_message
            }
          ]
        }
      ],
      response_format={
        "type": "json_object"
      },
      temperature=temperature,
      max_completion_tokens=2048,
      top_p=top_p,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

user_message에 지시사항을 적어도된다!

In [ ]:
def job_interveiw(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당 분야의 으뜸가는 면접관입니다.
    매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.
    """

    user_message = f"""
    << 지시사항 >>
    - 사용자가 제출한 채용공고의 내용을 바탕으로 예상 면접 질문과 답변을 작성해 주세요.
    - 하드 스킬과 소프트 스킬 두개의 섹션으로 나누어 작성해주세요.
    - 각 스킬별로 질문/답변을 3개씩 만들어주세요.

    << 출력형식 >>
    출력은 json 형식으로만 반환되어야 합니다.

    {{
      "hard_skill": [
        {{
          "question": "",
          "anser": "",
        }}
      ],
      "soft_skill": [
        {{
          "question": "",
          "anser": "",
        }}
      ]
    }}

    << 채용공고 >>
    {job_posting}
    """

    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": system_message
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": user_message
            }
          ]
        }
      ],
      response_format={
        "type": "json_object"
      },
      temperature=temperature,
      max_completion_tokens=2048,
      top_p=top_p,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response.choices[0].message.content

In [24]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interveiw(job_posting)
print(job_interveiw(job_posting))

{
  "hard_skill": [
    {
      "question": "Python 프로그래밍을 활용한 머신러닝 시스템의 구축 경험에 대해 설명해 주세요.",
      "anser": "최근 프로젝트에서 Python을 사용하여 머신러닝 모델을 구축한 경험이 있습니다. TensorFlow와 scikit-learn을 이용해 데이터를 전처리하고 모델을 학습시켰습니다. 해당 모델은 Flask를 통해 RESTful API로 배포하였으며, 성능을 모니터링하기 위해 Grafana와 Prometheus를 사용했습니다."
    },
    {
      "question": "Docker와 Kubernetes를 활용한 컨테이너 오케스트레이션 경험을 설명해 주세요.",
      "anser": "저는 Docker를 사용하여 애플리케이션을 컨테이너화하고, Kubernetes 클러스터를 구축하여 이를 배포한 경험이 있습니다. Helm을 사용하여 배포 관리를 간편하게 했으며, 스케일링 및 로드 밸런싱을 통해 트래픽 증가에 대한 대응을 하였습니다."
    },
    {
      "question": "CI/CD 도구(Jenkins, GitLab 등)를 사용하여 DevOps 환경에서의 배포 경험에 대해 말씀해 주세요.",
      "anser": "Jenkins를 사용하여 CI/CD 파이프라인을 구축하였습니다. 코드가 GitLab에 커밋될 때마다 자동으로 빌드, 테스트, 배포 프로세스가 진행되도록 설정했고, 이를 통해 배포 오류를 미리 방지하고 운영 효율성을 극대화했습니다."
    }
  ],
  "soft_skill": [
    {
      "question": "팀원들과의 협업을 통해 문제를 해결한 사례를 말씀해 주세요.",
      "anser": "팀 프로젝트에서 특정 모델의 성능이 기대 이하로 나오는 문제가 있었습니다. 이를 해결하기 위해 데이터 사이언티스트와 함께 회의를 진행하여 데이터 품질 문제를 발견하고, 새로운 데이터 전처리 방식을 적용하여 

In [27]:
import json

data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'Python을 사용하여 머신러닝 모델을 배포하기 위한 기본적인 흐름을 설명해주세요.', 'anser': '먼저, 모델을 학습시키기 위해 데이터를 준비하고 전처리합니다. 그 다음, 모델을 훈련시키고 최적의 하이퍼파라미터를 찾습니다. 모델이 준비되면, Flask나 FastAPI와 같은 웹 프레임워크를 활용해 API 형태로 배포합니다. 또한, Docker를 사용해 컨테이너화하고, Kubernetes를 통해 오케스트레이션을 관리합니다. 마지막으로, CI/CD 파이프라인을 구축해 자동화된 테스트 및 배포 과정을 구현합니다.'}, {'question': 'Kubernetes와 Docker를 사용하여 머신러닝 인프라를 구축하는 과정에 대해 설명해주세요.', 'anser': '우선, Docker를 사용해 각 ML 구성 요소를 컨테이너화합니다. 예를 들어, 데이터 처리, 모델 훈련, 예측 API 등을 각각 별도의 컨테이너로 만듭니다. 이후, Kubernetes에서 이 컨테이너들을 Pod로 배포하여 클러스터에서 관리할 수 있게 합니다. 서비스 디스커버리, 로드 밸런싱 및 자동 확장과 같은 Kubernetes의 기능을 활용해 인프라를 효율적으로 운영합니다.'}, {'question': 'MLflow를 사용하여 모델을 서빙하는 방법에 대해 설명해주세요.', 'anser': 'MLflow는 모델 관리 및 배포를 위한 유용한 툴입니다. 먼저, 모델을 MLflow Tracking에 로그하여 실험을 관리합니다. 모델이 준비되면, MLflow Models를 사용하여 REST API로 모델을 서빙합니다. `mlflow models serve` 명령어를 통해 쉽게 시작할 수 있으며, 이 API 호출을 통해 실시간으로 예측을 수행할 수 있습니다. 또한, 모델 버전을 관리하고, 다양한 플랫폼에 배포할 수 있는 기능도 제공받습니다.'}]
[{'question': '팀원 간의 문제 해결을 위해 어떻게 협업하실 건가요?', 'anser': '저는 팀